In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict

# 1. 네이버 뉴스 검색 결과에서 기사 링크 수집 (제목 필터링 제거, 최대 50개 확장)
def search_naver_news_links(keyword, max_results=50):
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
            '(KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
        )
    }

    collected = []
    page = 1

    while len(collected) < max_results:
        start = (page - 1) * 10 + 1
        search_url = f"https://search.naver.com/search.naver?where=news&query={keyword}&start={start}"
        res = requests.get(search_url, headers=headers)

        if res.status_code != 200:
            print(f"⚠️ 페이지 {page} 요청 실패 (코드 {res.status_code})")
            break

        soup = BeautifulSoup(res.text, 'lxml')
        link_tags = soup.select('a.info')

        count_this_page = 0
        for a in link_tags:
            if a.get_text() == "네이버뉴스":
                link = a['href']
                title = a.find_previous('a').get_text()
                if (title, link) not in collected:
                    collected.append((title, link))
                    count_this_page += 1
            if len(collected) >= max_results:
                break

        if count_this_page == 0:
            break
        page += 1

    return collected

# 2. 기사 본문 크롤링
def get_naver_article_body(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    res = requests.get(url, headers=headers)
    if res.status_code != 200:
        return ""

    soup = BeautifulSoup(res.text, "lxml")
    content = soup.find("div", {"id": "newsct_article"}) or soup.find("div", {"class": "newsct_article"})
    return content.get_text(separator=" ").strip() if content else ""

# 3. 판매 수치 및 변화율 추출 함수 (변화율 계산 포함)
def extract_sales_changes_with_rate(text):
    results = []

    pattern_change = re.findall(
        r'(?P<brand>[가-힣A-Za-z]+).*?(?P<year1>20\d{2})년.*?(?P<sales1>[\d,만]+)대.*?(?P<year2>20\d{2})년.*?(?P<sales2>[\d,만]+)대',
        text
    )

    for brand, y1, s1, y2, s2 in pattern_change:
        s1_num = int(s1.replace(',', '').replace('만', '0000'))
        s2_num = int(s2.replace(',', '').replace('만', '0000'))

        rate = ((s2_num - s1_num) / s1_num) * 100
        direction = "증가" if rate > 0 else "감소"
        rate_str = f"{abs(round(rate, 1))}% {direction}"

        results.append({
            "브랜드": brand,
            "연도": y1,
            "판매대수": s1_num,
            "변화율": None
        })
        results.append({
            "브랜드": brand,
            "연도": y2,
            "판매대수": s2_num,
            "변화율": rate_str
        })

    return results

# 4. 실행
def main():
    search_keyword = input("🔍 네이버 뉴스에서 검색할 키워드를 입력하세요: ")
    print(f"\n[🔎 '{search_keyword}' 로 뉴스 검색 중...]\n")

    news_list = search_naver_news_links(search_keyword, max_results=50)
    all_sales_data = []

    for idx, (title, link) in enumerate(news_list, 1):
        print(f"[{idx}] 📰 {title}")
        print(f"     🔗 {link}")
        body = get_naver_article_body(link)
        if not body:
            print("     ⚠️ 본문 수집 실패\n")
            continue

        sales_data = extract_sales_changes_with_rate(body)
        for item in sales_data:
            item['출처번호'] = idx  # 기사 번호 연결
        all_sales_data.extend(sales_data)
        print("     ✅ 수치 정보 추출 완료\n")

    print("\n📊 최종 판매 수치 및 변화율 정보 (기사 번호 매칭):\n")
    for item in all_sales_data:
        print(f"[{item['출처번호']}] 📌 브랜드: {item['브랜드']}, 연도: {item['연도']}, 판매대수: {item['판매대수']}, 변화율: {item['변화율']}")

if __name__ == "__main__":
    main()
